In [ ]:
import torch
import torch.autograd
from torch.autograd import Variable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# loading 60.000 rows of training data from MNIST
df_mnist = pd.read_csv(u'data/mnist_train.csv', sep=',', header=None)

# distributing the data as features & target
features = np.array(df_mnist.iloc[:, [i+1 for i in range(784)]])
target = np.array(df_mnist.iloc[:, 0])

In [ ]:
# N is batch size, getting N rows in every iteration; D_in is input dimension 28x28=784;
# H is hidden dimension, we decide; D_out is output dimension, which is 1.
N, D_in, H, D_out = 100, 784, 28, 1

# defining the functions to be used between our layers
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# defining the loss function
loss_fn = torch.nn.CrossEntropyLoss(reduce=True, size_average=False)

# eta value of this regression
learning_rate = 1e-4

# defining optimizer algorithm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# starting epochs
for t in range(600):
    # selecting N rows for every epoch
    idx = np.random.choice(len(x), size=N, replace=True)
    
    # fetching the selected data as x & y
    x = torch.FloatTensor(features[idx,:])
    y = torch.LongTensor(target[idx])
    
    # calculating y_pred by passing fetched x to our model
    y_pred = model(x)

    # comparing calculated y_pred and actual y
    loss = loss_fn(y_pred, y)
    
    # printing the loss in every 100 epochs
    if(t%100 == 0):
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

In [ ]:
# now, our model is trainde, it is time for 10.000 rows of test data
df_mnist_test = pd.read_csv(u'mnist_test.csv', sep=',', header=None)

# distributing the test data as features_test & target_test
x_test = torch.FloatTensor(np.array(df_mnist_test.iloc[:, [i+1 for i in range(784)]]))
y_test = torch.LongTensor(np.array(df_mnist_test.iloc[:, 0]))

# getting the result of the test data from the trained model
result = torch.round(model(x_test))
        
# building a confusion matrix
confusion_matrix = torch.zeros((10, 10))

for i in range(len(result)):
    confusion_matrix[int(y_test[i]), int(result[i])] += 1
    
print("Confusion matrix:\n", confusion_matrix, "\n")
    
accuracy = torch.sum(torch.diag(confusion_matrix))/torch.sum(confusion_matrix)

print("Accuracy:", accuracy*100, "%\n")